In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from preprocess import listFiles
from register import loadData
from scipy.spatial.transform import Rotation as R

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [209]:
def quat2RotationMatrix(q):
    """
    Convert a quaternion into a rotation matrix.
    
    Args:
    q (numpy.ndarray): A 4-element array representing the quaternion (w, x, y, z)
    
    Returns:
    numpy.ndarray: A 3x3 rotation matrix
    """
    w, x, y, z = q

    # Compute the rotation matrix elements
    R = np.array([
        [1 - 2*y*y - 2*z*z, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x*x - 2*z*z, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x*x - 2*y*y]
    ])
    return R


def transformPointCloud(point_cloud, translation, quaternion):
    """
    Apply translation and rotation to a point cloud.
    
    Args:
        point_cloud (np.ndarray): An n x 3 array representing the point cloud.
        translation (np.ndarray): A 1 x 3 array representing the translation vector.
        quaternion (np.ndarray): A 1 x 4 array representing the quaternion for rotation.
    
    Returns:
        np.ndarray: The transformed point cloud.
    """
    # Validate inputs
    assert point_cloud.shape[1] == 3, "Point cloud must be an n x 3 array."
    assert translation.shape == (3,), "Translation must be a 1 x 3 array."
    assert quaternion.shape == (4,), "Quaternion must be a 1 x 4 array."

    R = quat2RotationMatrix(quaternion)
    R_t = np.transpose(R)
    # R = np.eye(3)
    temp = R@(np.transpose(point_cloud))
    temp2 = np.transpose(temp) + translation

    return temp2


def quickVizReg(source, target):
    pcd1 = o3d.geometry.PointCloud()
    pcd2 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(source)
    pcd2.points = o3d.utility.Vector3dVector(target)
    pcd1.paint_uniform_color([1, 0, 0])
    pcd2.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([pcd1,pcd2])

def quickVisualize(points):
    """ Visualize pointcloud from numpy array"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])
    return

In [3]:
path = r'datasets/pose/box_plant2_pose.json'
with open(path,'r') as file:
    pose = json.load(file)

path = r'datasets/json/box_plant2/box_plant2_frame0_100.json'
with open(path,'r') as file:
    points = json.load(file)
    

In [38]:
pose_keys = list(pose.keys())
point_keys = list(points.keys())

In [203]:
frame1_idx = 0
frame2_idx = 70

points1 = np.array(points[point_keys[frame1_idx]])
points2 = np.array(points[point_keys[frame2_idx]])

translation1 = np.array(pose[pose_keys[frame1_idx]]['translation'])
translation2 = np.array(pose[pose_keys[frame2_idx]]['translation'])
rotation1 = pose[pose_keys[frame1_idx]]['rotation']
rotation2 = pose[pose_keys[frame2_idx]]['rotation']

rotation2 = [0.9430642707686508,
 -0.034828161526489224,
 -0.07748553691075596,
 0.32157856261562184,
 ]

rotation1 = [0.9997741342501786,
    -0.013439967902957725,
 -0.015594347893096654,
 0.0052786419524853976]

In [210]:
points1_reg = transformPointCloud(points1, np.array(translation1), np.array(rotation1))
points2_reg = transformPointCloud(points2, np.array(translation2), np.array(rotation2))

In [211]:
quickVizReg(points1_reg, points2_reg)

In [46]:
quickVizReg(points1, points2)